# 🕸️ Social Network Analysis (SNA) dari Data Twitter
Notebook ini memvisualisasikan dan menganalisis jaringan interaksi dari data yang sudah di-scrape.

In [ ]:
# Install library yang dibutuhkan
!pip install pandas networkx matplotlib seaborn pyvis -q

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from pyvis.network import Network
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ── Load data ──────────────────────────────────────────────────────────────
DATA_DIR = Path('../data')

# Ambil file terbaru
nodes_files = sorted(DATA_DIR.glob('nodes_*.csv'), key=lambda f: f.stat().st_mtime, reverse=True)
edges_files = sorted(DATA_DIR.glob('edges_*.csv'), key=lambda f: f.stat().st_mtime, reverse=True)

nodes_file = nodes_files[0]
edges_file = edges_files[0]

print(f'Nodes: {nodes_file}')
print(f'Edges: {edges_file}')

nodes_df = pd.read_csv(nodes_file)
edges_df = pd.read_csv(edges_file)

print(f'\n📊 Jumlah nodes: {len(nodes_df)}')
print(f'📊 Jumlah edges: {len(edges_df)}')

nodes_df.head()

In [ ]:
edges_df.head()

## 🏗️ Membangun Graph

In [ ]:
# Buat directed graph
G = nx.DiGraph()

# Tambah nodes dengan atribut
for _, row in nodes_df.iterrows():
    G.add_node(row['id'],
               label=row.get('label', row['id']),
               followers=row.get('followers', 0),
               tweet_count=row.get('tweet_count', 0))

# Tambah edges
for _, row in edges_df.iterrows():
    G.add_edge(row['source'], row['target'],
               weight=row.get('weight', 1),
               edge_type=row.get('type', 'interaction'))

print(f'Graph berhasil dibuat!')
print(f'  Nodes : {G.number_of_nodes()}')
print(f'  Edges : {G.number_of_edges()}')
print(f'  Density: {nx.density(G):.4f}')

## 📐 Centrality Metrics

In [ ]:
# Hitung centrality
degree_centrality    = nx.degree_centrality(G)
in_degree_centrality = nx.in_degree_centrality(G)
betweenness          = nx.betweenness_centrality(G, k=min(100, len(G)))
pagerank             = nx.pagerank(G, alpha=0.85)

centrality_df = pd.DataFrame({
    'user':        list(degree_centrality.keys()),
    'degree':      list(degree_centrality.values()),
    'in_degree':   list(in_degree_centrality.values()),
    'betweenness': list(betweenness.values()),
    'pagerank':    list(pagerank.values()),
}).sort_values('pagerank', ascending=False)

print('🏆 Top 15 Influencer (berdasarkan PageRank):')
centrality_df.head(15)

In [ ]:
# Simpan hasil centrality
centrality_df.to_csv(DATA_DIR / 'centrality_results.csv', index=False)
print('✅ Disimpan ke data/centrality_results.csv')

## 🎨 Visualisasi Graph (Static)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 12))
ax.set_facecolor('#0d1117')
fig.patch.set_facecolor('#0d1117')

# Filter node kecil agar graph tidak terlalu padat
top_nodes = centrality_df.head(80)['user'].tolist()
G_sub = G.subgraph(top_nodes)

pos = nx.spring_layout(G_sub, k=2, seed=42)

# Node size = pagerank
node_size = [pagerank.get(n, 0.001) * 15000 + 50 for n in G_sub.nodes()]

# Node color = in-degree
node_color = [in_degree_centrality.get(n, 0) for n in G_sub.nodes()]

nx.draw_networkx_edges(G_sub, pos, ax=ax,
    alpha=0.3, width=0.5, edge_color='#4a9eff', arrows=True,
    arrowsize=8, connectionstyle='arc3,rad=0.1')

sc = nx.draw_networkx_nodes(G_sub, pos, ax=ax,
    node_size=node_size, node_color=node_color,
    cmap=cm.plasma, alpha=0.9)

# Label hanya untuk top 20
top20 = centrality_df.head(20)['user'].tolist()
labels = {n: n for n in G_sub.nodes() if n in top20}
nx.draw_networkx_labels(G_sub, pos, labels, ax=ax,
    font_size=7, font_color='white', font_weight='bold')

plt.colorbar(sc, ax=ax, label='In-Degree Centrality')
ax.set_title('Twitter Interaction Network', color='white', fontsize=16, pad=15)
ax.axis('off')
plt.tight_layout()
plt.savefig(DATA_DIR / 'network_visualization.png', dpi=150,
            bbox_inches='tight', facecolor='#0d1117')
plt.show()
print('✅ Graph disimpan ke data/network_visualization.png')

## 🌐 Visualisasi Interaktif (HTML)

In [ ]:
net = Network(height='750px', width='100%', bgcolor='#0d1117',
               font_color='white', directed=True)

G_vis = G.subgraph(centrality_df.head(80)['user'].tolist())

for node in G_vis.nodes():
    size = pagerank.get(node, 0.001) * 2000 + 10
    net.add_node(node, label=node, size=size,
                 title=f'@{node}\nPageRank: {pagerank.get(node,0):.4f}\nIn-Degree: {in_degree_centrality.get(node,0):.4f}',
                 color='#4a9eff')

for src, tgt, data in G_vis.edges(data=True):
    net.add_edge(src, tgt, width=data.get('weight', 1) * 0.5,
                 title=data.get('edge_type', ''))

net.set_options('''
  var options = {
    "physics": {
      "forceAtlas2Based": {
        "gravitationalConstant": -50,
        "centralGravity": 0.01,
        "springLength": 100
      },
      "solver": "forceAtlas2Based"
    }
  }
''')

output_html = str(DATA_DIR / 'network_interactive.html')
net.save_graph(output_html)
print(f'✅ Visualisasi interaktif disimpan ke {output_html}')

## 🔍 Community Detection

In [ ]:
# Deteksi komunitas menggunakan Louvain (undirected)
import community as community_louvain

G_undirected = G_vis.to_undirected()
partition = community_louvain.best_partition(G_undirected)

n_communities = len(set(partition.values()))
print(f'Jumlah komunitas ditemukan: {n_communities}')

# Gabungkan dengan centrality
centrality_df['community'] = centrality_df['user'].map(partition)
centrality_df.head(20)

In [ ]:
# Simpan semua hasil
centrality_df.to_csv(DATA_DIR / 'centrality_with_community.csv', index=False)
print('✅ Analisis selesai! File tersimpan di folder data/')
print('   - centrality_with_community.csv')
print('   - network_visualization.png')
print('   - network_interactive.html')